In [1]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT license.

from typing import List, Optional, Tuple, Dict

from maro.simulator import Env
from maro.simulator.scenarios.oncall_routing import Coordinate
from maro.simulator.scenarios.oncall_routing.common import Action, PostponeAction, OncallRoutingPayload, AllocateAction
from maro.simulator.scenarios.oncall_routing.order import OrderStatus
from maro.utils import set_seeds
import pandas as pd
from greedy2 import _get_actions
# from greedy_with_time_window import _get_actions
# from greedy import _get_actions

from examples.oncall_routing.utils import refresh_segment_index

set_seeds(2048)

oncall_id_list = []

def order_status2_str(order_status):
    if order_status == OrderStatus.NOT_READY:
        return "NotReady"
    elif order_status == OrderStatus.READY_IN_ADVANCE:
        return "ReadyInAdvance"
    elif order_status == OrderStatus.IN_PROCESS:
        return "InProcess"
    elif order_status == OrderStatus.IN_PROCESS_BUT_DELAYED:
        return "Delayed"
    elif order_status == OrderStatus.COMPLETED:
        return "Complete"
    elif order_status == OrderStatus.TERMINATED:
        return "Terminated"
    else:
        return "Dummy"

def collect_info(running_env: Env, event: OncallRoutingPayload):
    result_list = []
    tick = running_env.tick
    route_meta_info_dict = running_env.business_engine._get_route_meta_info_dict(tick)
    carriers_in_stop: List[bool] = (running_env.snapshot_list["carriers"][tick::"in_stop"] == 1).tolist()
    carrier_position_dict: Dict[str, Tuple[bool, int, int, Coordinate]] = {
        route_name: [
            carriers_in_stop[meta["carrier_idx"]],
            meta["estimated_duration_to_the_next_stop"],
            meta["estimated_next_departure_tick"],
            meta["current_staying_stop_coordinate"]
        ]
        for route_name, meta in route_meta_info_dict.items()
    }
    finished_orders = running_env.business_engine.get_finished_orders_dict()
    for route_name in finished_orders.keys():
        meta = route_meta_info_dict[route_name]
        for order, finished_tick in finished_orders[route_name]:
            result = [tick, route_name, 'COMPLETED', 
                      order.coord[0], 
                      order.coord[1], 
                      finished_tick,
                      order.open_time,
                      order.close_time,
                      order.is_delivery,
                      order_status2_str(order._status),
                      meta['carrier_idx'],
                      (-1 if carrier_position_dict[route_name][3] is None else carrier_position_dict[route_name][3][0]),
                      (-1 if carrier_position_dict[route_name][3] is None else carrier_position_dict[route_name][3][1]),
                      order.id
                     ]
            result_list.append(result)
    if event:
        oncall_orders = event.oncall_orders
        route_plan_dict = event.route_plan_dict
        for route_name, meta in route_meta_info_dict.items():
            planned_orders = route_plan_dict[route_name]
            for i, planned_order in enumerate(planned_orders):
                result = [tick, route_name, 'PLANNED', 
                        planned_order.coord[0], 
                        planned_order.coord[1], 
                        tick,
                        planned_order.open_time,
                        planned_order.close_time,
                        planned_order.is_delivery,
                        order_status2_str(planned_order.get_status(tick, env.business_engine._config.order_transition)),
                        meta['carrier_idx'],
                        (-1 if carrier_position_dict[route_name][3] is None else carrier_position_dict[route_name][3][0]),
                        (-1 if carrier_position_dict[route_name][3] is None else carrier_position_dict[route_name][3][1]),
                        planned_order.id
                        ]
                result_list.append(result)
        for oncall_order in oncall_orders:
            oncall_id_list.append(oncall_order.id)
            result = [tick, "Not Assigned", 'ONCALL', 
                    oncall_order.coord[0], 
                    oncall_order.coord[1], 
                    tick,
                    oncall_order.open_time,
                    oncall_order.close_time,
                    oncall_order.is_delivery,
                    order_status2_str(oncall_order.get_status(tick, env.business_engine._config.order_transition)),
                    -1,
                    -1,
                    -1,
                    oncall_order.id
                    ]
            result_list.append(result)
    df_orders = pd.DataFrame(data=result_list, 
                             columns=['tick', 'route_name', 'type',
                                      'order_lat', 'order_lng', 'finish_tick', 
                                      'open_time', 'close_time',
                                      'is_delivery', 'order_status',
                                      'carrier_id', 'carrier_lat', 'carrier_lng', 'order_id'])
    # df_orders.to_csv('hist_orders.csv', index=False)
    return df_orders


# Greedy: assign each on-call order to the closest stop on existing route.
if __name__ == "__main__":
    env = Env(
        scenario="oncall_routing", topology="example", start_tick=400, durations=1000,
    )
    print(env._business_engine._max_tick)
    print(env._business_engine._config.order_transition)
    env.reset(keep_seed=True)
    metrics, decision_event, is_done = env.step(None)
    df_list = []
    while not is_done:
        assert isinstance(decision_event, OncallRoutingPayload)
        df_orders = collect_info(env, decision_event)
        df_list.append(df_orders)
        print(f"Processing {len(decision_event.oncall_orders)} oncall orders at tick {env.tick}.")
        # metrics, decision_event, is_done = env.step(_get_actions(env, decision_event))
        # for oncall_order in decision_event.oncall_orders:
        metrics, decision_event, is_done = env.step(_get_actions(env, decision_event))
    df_orders = collect_info(env, None)
    df_list.append(df_orders)
    df = pd.concat(df_list)
    df.loc[:, 'type'] = df.apply(lambda row: ("ONCALL" if row['order_id'] in oncall_id_list else "PLANNED"), axis=1)
    df.to_csv("hist_routing_orders.csv", index=False)
    print(env.tick)  
    print(env.metrics) 

Config path: /Users/songlei/Library/CloudStorage/OneDrive-Personal/MSRA/Research/codes/smartMARO/maro/simulator/scenarios/oncall_routing/topologies/example
Loading oncall orders.
oncall 338
Loading plans.
Loading routes data from /Users/songlei/Library/CloudStorage/OneDrive-Personal/MSRA/Research/codes/smartMARO/maro/simulator/scenarios/oncall_routing/topologies/example/routes.csv.
Loading finished. Loaded data of 117 routes.
1420
{'buffer_before_open_time': 30, 'buffer_after_open_time': 30, 'last_tick_for_order_processing': 1400, 'processing_time': 1, 'processing_proportion_to_quantity': False}
oncall 338
Processing 1 oncall orders at tick 286.
Processing 1 oncall orders at tick 318.
Processing 1 oncall orders at tick 321.
Processing 1 oncall orders at tick 339.
Processing 1 oncall orders at tick 353.
Processing 1 oncall orders at tick 364.
Processing 1 oncall orders at tick 371.
Processing 2 oncall orders at tick 374.
Processing 1 oncall orders at tick 376.
Processing 2 oncall orders

In [5]:
df_cnt = df[((df['tick']==1419) & (df['type'] == "ONCALL"))].groupby('route_name').count().reset_index()
df_cnt.sort_values(by='order_id', inplace=True, ascending=False)
df_cnt.head()

,route_name,tick,type,order_lat,order_lng,finish_tick,open_time,close_time,is_delivery,order_status,carrier_id,carrier_lat,carrier_lng,order_id
0,528,335,335,335,335,335,335,335,335,335,335,335,335,335
1,649,1,1,1,1,1,1,1,1,1,1,1,1,1
2,665,1,1,1,1,1,1,1,1,1,1,1,1,1
3,927,1,1,1,1,1,1,1,1,1,1,1,1,1


In [7]:
df[((df['tick']==1419) & (df['route_name'] == "528") & (df['type'] == "ONCALL"))]

,tick,route_name,type,order_lat,order_lng,finish_tick,open_time,close_time,is_delivery,order_status,carrier_id,carrier_lat,carrier_lng,order_id
1963,1419,528,ONCALL,32.78989,-117.25326,460,480,600,False,Complete,35,-1.0,-1.0,oncall_0156
1964,1419,528,ONCALL,32.65090,-116.96133,471,480,810,False,Complete,35,-1.0,-1.0,oncall_0062
1965,1419,528,ONCALL,32.62957,-116.96505,540,570,720,False,Complete,35,-1.0,-1.0,oncall_0097
1966,1419,528,ONCALL,32.65221,-116.96316,630,660,900,False,Complete,35,-1.0,-1.0,oncall_0277
1967,1419,528,ONCALL,32.62127,-116.96610,639,588,708,False,Complete,35,-1.0,-1.0,oncall_0090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2306,1419,528,ONCALL,32.74262,-116.97233,1412,810,1260,False,Terminated,35,-1.0,-1.0,oncall_0084
2309,1419,528,ONCALL,32.73401,-117.05698,1417,840,1380,False,Terminated,35,-1.0,-1.0,oncall_0069
2310,1419,528,ONCALL,32.73255,-117.05835,1417,660,840,False,Terminated,35,-1.0,-1.0,oncall_0229
2311,1419,528,ONCALL,32.72220,-117.07204,1417,600,721,False,Terminated,35,-1.0,-1.0,oncall_0094
